## Clause Identification

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")
pd.set_option('display.max_colwidth', None)  # Ensure no truncation of column content
pd.set_option('display.max_rows', None)     # Display all rows
pd.set_option('display.max_columns', None) 

e:\DS340\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "Datasets/master_clauses.csv"
df = pd.read_csv(file_path)
df['Filename'] = df['Filename'].str.replace('.pdf', '.txt', regex=False)

## Tokenizing Sentence for Terms and Conditions

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import ast

# Step 1: Ensure all column values are lists
def ensure_list(value):
    if isinstance(value, list):
        return value  # Already a list
    elif isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        # If it's a string that looks like a list, try to parse it
        try:
            import ast
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return [value]  # If parsing fails, wrap it as a single-item list
    elif pd.notnull(value):
        return [value]  # If it's a scalar, wrap it as a single-item list
    else:
        return []  # Handle NaN or None values as empty lists


# Combine text columns to form the input for tokenization
columns_to_include = [
    'Governing Law',
    'Non-Compete',
    'Exclusivity',
    'Non-Disparagement',
    'No-Solicit Of Customers',
    'No-Solicit Of Employees',
]

# Load the ContractBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

df_to_flatten = df[columns_to_include]


for column in df_to_flatten.columns:
    df_to_flatten[column] = df_to_flatten[column].apply(ensure_list)

# Step 2: Flatten the dataframe
flattened_rows = []
for index, row in df_to_flatten.iterrows():
    for column in df_to_flatten.columns:
        for clause in row[column]:
            flattened_rows.append({'Category': column, 'text': clause})

flattened_df = pd.DataFrame(flattened_rows)
# Flatten the "Clause" column: Convert lists into plain strings
flattened_df['text'] = flattened_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
# Remove rows with empty "Clause"
flattened_df = flattened_df[flattened_df['text'].str.strip() != ""]
# Create a new dataframe
df_positives = pd.DataFrame(flattened_df['text'])

df_positives['labels'] = 1

# Display the flattened dataframe
df_positives.head(5)


C:\Users\Josh_\AppData\Local\Temp\ipykernel_3512\3403944320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_flatten[column] = df_to_flatten[column].apply(ensure_list)


,text,labels
0,"This Agreement is accepted by Company in the State of Nevada and shall be governed by and construed in accordance with the laws thereof, which laws shall prevail in the event of any conflict.",1
1,"Company shall not specify the business practices of MA, nor regulate the manner in which MA shall operate its business, provided that MA (a) conducts business in a manner that reflects favorably at all times on the Technology sold and the good name, goodwill and reputation of Company and its affiliates<omitted>",1
2,"This Agreement is subject to all laws, regulations, license conditions and decisions of the Canadian Radio-television and Telecommunications Commission (""CRTC"") municipal, provincial and federal governments or other authorities which are applicable to Rogers and/or Licensor, and which are now in force or hereafter adopted (""Applicable Law"").",1
3,This Agreement shall be governed by laws of the Province of Ontario and the federal laws of Canada applicable therein.,1
4,"All questions with respect to the construction of this Agreement, and the rights and liabilities of the Parties hereto, shall be governed by the laws of the State of Florida.",1


## Columns to train against (the non positive labels)

In [31]:
exclude_columns = ['Parties','Agreement Date','Effective Date', 'Expiration Date', "Filename", "Document Name", "Document Name-Answer"]
dynamic_exclude_columns = [col for col in df.columns if "-Answer" in col]
all_columns_to_exclude = set(exclude_columns + dynamic_exclude_columns+columns_to_include)
df_negatives= df.drop(columns=all_columns_to_exclude)


for column in df_negatives.columns:
    df_negatives[column] = df_negatives[column].apply(ensure_list)

# Step 2: Flatten the dataframe
flattened_rows = []
for index, row in df_negatives.iterrows():
    for column in df_negatives.columns:
        for clause in row[column]:
            flattened_rows.append({'Category': column, 'text': clause})

flattened_df = pd.DataFrame(flattened_rows)
# Flatten the "Clause" column: Convert lists into plain strings
flattened_df['text'] = flattened_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
# Remove rows with empty "Clause"
flattened_df = flattened_df[flattened_df['text'].str.strip() != ""]

# Create a new dataframe
df_negatives = pd.DataFrame(flattened_df['text'])

df_negatives['labels'] = 0

# Display the flattened dataframe
df_negatives.columns

Index(['text', 'labels'], dtype='object')

## Merging the datasets after labelling

In [68]:
# Merge the dataframes
merged_df = pd.concat([df_positives, df_negatives], ignore_index=True)
merged_df = merged_df[['text','labels']]



In [44]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
# Define a tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],        # The text column to tokenize
        padding="max_length",    # Pad all sequences to the same maximum length
        truncation=True,         # Truncate sequences longer than the model's limit
        max_length=128           # Maximum sequence length for BERT-based models
    )


In [45]:
from datasets import Dataset

# Example dataset
import pandas as pd

# # Convert dataframe to Hugging Face Dataset
# hf_dataset = Dataset.from_pandas(merged_df)


## Train Test Splits

In [46]:
# # Drop rows with missing text values
# hf_dataset = hf_dataset.filter(lambda example: example["text"] is not None) #already do this....

In [84]:
from sklearn.model_selection import train_test_split
# # Split into train and test sets using sklearn
# train_dataset, test_dataset = train_test_split(merged_df, test_size=0.2, random_state=42)

In [ ]:
# # Apply the tokenizer to the dataset separetely
# tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# # Remove the original text column since it's no longer needed
# tokenized_dataset = tokenized_dataset.remove_columns(["text"])


Map: 100%|██████████| 8783/8783 [00:01<00:00, 6793.89 examples/s]


## Train and Test Splits

In [ ]:
import random
# train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)
# Step 1: Randomly split the dataset into train and test
random.seed(42)  # Set seed for reproducibility
train_indices = random.sample(range(len(df)), int(0.8 * len(df)))  # 80% train
test_indices = list(set(range(len(df))) - set(train_indices))      # Remaining 20% test

train_df = merged_df.iloc[train_indices].reset_index(drop=True)
test_df = merged_df.iloc[test_indices].reset_index(drop=True)

In [ ]:
train_df.head()

In [91]:
from transformers import pipeline
from datasets import Dataset
import pandas as pd

# Load the paraphrasing pipeline
paraphraser = pipeline("text2text-generation", model="t5-small", device=0)

# Define a function for paraphrasing
def paraphrase_batch(batch, num_return_sequences=3, num_beams=3):
    """
    Paraphrase a batch of texts using the Hugging Face pipeline.
    """
    inputs = [f"paraphrase: {text}" for text in batch['text']]
    paraphrased_results = paraphraser(
        inputs,
        max_length=128 ,
        num_return_sequences=num_return_sequences,
        num_beams=num_beams,
        truncation=True
    )

    paraphrased_texts = []
    # Process the results to extract 'generated_text'
    if isinstance(paraphrased_results, list) and isinstance(paraphrased_results[0], list):
        # Handle batched results: `paraphrased_results` is a nested list
        for sublist in paraphrased_results:
            paraphrased_texts.extend([result['generated_text'] for result in sublist])
    elif isinstance(paraphrased_results, list):
        # Handle unbatched results: `paraphrased_results` is a flat list
        paraphrased_texts.extend([result['generated_text'] for result in paraphrased_results])
    else:
        raise ValueError("Unexpected paraphrased_results structure.")

    # Repeat labels for each paraphrased result
    labels = sum([[label] * num_return_sequences for label in batch['labels']], [])

    return {'text': paraphrased_texts, 'labels': labels}

# Convert your DataFrame to a Dataset
dataset = Dataset.from_pandas(train_df)

# Apply the paraphrasing function in batches
batch_size = 16  # Adjust based on your GPU capacity
paraphrased_dataset = dataset.map(
    lambda batch: paraphrase_batch(batch, num_return_sequences=3),
    batched=True,
    batch_size=batch_size
)

df_augmented = pd.DataFrame({
    'text': [item for sublist in paraphrased_dataset['text'] for item in sublist] if isinstance(paraphrased_dataset['text'][0], list) else paraphrased_dataset['text'],
    'labels': [item for sublist in paraphrased_dataset['labels'] for item in sublist] if isinstance(paraphrased_dataset['labels'][0], list) else paraphrased_dataset['labels']
})


# Append augmented data to the original DataFrame
df_combined = pd.concat([train_df, df_augmented], ignore_index=True)

# Display the combined dataset
print(df_combined.head())
train_df = df_combined

Map:   0%|          | 0/408 [00:00<?, ? examples/s]


KeyError: 'text'

In [82]:
# Split the dataset into training and validation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.3)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

# Check the number of samples
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Training samples: 6148
Validation samples: 2635


## Augment the Train Dataset

In [ ]:
from transformers import pipeline
from datasets import Dataset
import pandas as pd

# Load the paraphrasing pipeline
paraphraser = pipeline("text2text-generation", model="t5-small", device=0)

# Define a function for paraphrasing
def paraphrase_batch(batch, num_return_sequences=3, num_beams=3):
    """
    Paraphrase a batch of texts using the Hugging Face pipeline.
    """
    inputs = [f"paraphrase: {text}" for text in batch['text']]
    paraphrased_results = paraphraser(
        inputs,
        max_length=128,
        num_return_sequences=num_return_sequences,
        num_beams=num_beams,
        truncation=True
    )

    paraphrased_texts = []
    # Process the results to extract 'generated_text'
    if isinstance(paraphrased_results, list) and isinstance(paraphrased_results[0], list):
        # Handle batched results: `paraphrased_results` is a nested list
        for sublist in paraphrased_results:
            paraphrased_texts.extend([result['generated_text'] for result in sublist])
    elif isinstance(paraphrased_results, list):
        # Handle unbatched results: `paraphrased_results` is a flat list
        paraphrased_texts.extend([result['generated_text'] for result in paraphrased_results])
    else:
        raise ValueError("Unexpected paraphrased_results structure.")

    # Repeat labels for each paraphrased result
    labels = sum([[label] * num_return_sequences for label in batch['labels']], [])

    return {'text': paraphrased_texts, 'labels': labels}

# Convert your DataFrame to a Dataset
dataset = Dataset.from_pandas(merged_df)

# Apply the paraphrasing function in batches
batch_size = 16  # Adjust based on your GPU capacity
paraphrased_dataset = dataset.map(
    lambda batch: paraphrase_batch(batch, num_return_sequences=3),
    batched=True,
    batch_size=batch_size
)

  

df_augmented = pd.DataFrame({
    'text': [item for sublist in paraphrased_dataset['text'] for item in sublist] if isinstance(paraphrased_dataset['text'][0], list) else paraphrased_dataset['text'],
    'labels': [item for sublist in paraphrased_dataset['labels'] for item in sublist] if isinstance(paraphrased_dataset['labels'][0], list) else paraphrased_dataset['labels']
})


# Append augmented data to the original DataFrame
df_combined = pd.concat([merged_df, df_augmented], ignore_index=True)

# Display the combined dataset
print(df_combined.head())
merged_df = df_combined

Map:   0%|          | 0/7026 [00:18<?, ? examples/s]


TypeError: list indices must be integers or slices, not str

## Model Training

In [ ]:
print(tokenized_dataset[0])  # Inspect the first example


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load ContractBERT for binary classification (2 labels: 0 and 1)
model = AutoModelForSequenceClassification.from_pretrained(
    "nlpaueb/legal-bert-base-uncased",
    num_labels=2  # Binary classification
)
model = model.to("cuda")  # Explicitly move the model to GPU

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,                     # Evaluate every 500 steps
    save_steps=500,                     # Save every 500 steps
    learning_rate=3e-5,                 # Adjusted learning rate
    per_device_train_batch_size=16,     # Smaller batch size to fit GPU memory
    gradient_accumulation_steps=2,      # Effective batch size = 32
    per_device_eval_batch_size=16,      
    num_train_epochs=5,                 # More epochs for better training
    weight_decay=0.01,
    save_total_limit=4,
    fp16=True,                          # Mixed precision for better GPU utilization
    disable_tqdm=False,
    log_level="error",
    lr_scheduler_type="linear",         # Linear decay of learning rate
    warmup_steps=500,                   # Gradual learning rate warmup
    load_best_model_at_end=True,
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                       # Pretrained model
    args=training_args,                # Training arguments
    train_dataset=train_dataset,       # Training dataset
    eval_dataset=val_dataset           # Validation dataset
)
print(len(val_dataset))

In [ ]:
# Start fine-tuning the model
trainer.train()

In [ ]:

# Save the best model to a specific directory
save_directory = 'final_confidentiality_and_NDA'
# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
import evaluate
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Load the accuracy metric using the `evaluate` library
accuracy_metric = evaluate.load("accuracy")

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Get predicted class
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy["accuracy"]}

# Define TrainingArguments for evaluation
training_args = TrainingArguments(
    output_dir="./final_confidentiality_and_NDA",          # Directory to save results
    evaluation_strategy="epoch",    # Evaluate at the end of every epoch
    per_device_eval_batch_size=16,  # Batch size for evaluation
    logging_dir="./logs",           # Directory for logs
)


In [ ]:
# Load the saved model for evaluation
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("final_confidentiality_and_NDA")

# Create a Trainer object for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_dataset,  # Validation dataset
    compute_metrics=compute_metrics,
)

# Evaluate the model
results = trainer.evaluate()

# Print the validation accuracy
print(f"Validation Accuracy: {results['eval_accuracy']:.4f}")
